In [1]:
import os
import numpy as np

In [162]:
data_dir = '../data/morpho/nInputs_pscratch/'
data_files = os.listdir(data_dir)

data = {}

for data_file in data_files:
    seg_id = data_file.split('_')[2]
    n = int(data_file.split('_')[3].split('.')[0])
    spikes = np.load(f'{data_dir}{data_file}')
    
    if seg_id not in data:
        data[seg_id] = {}
    
    data[seg_id][n] = spikes

In [164]:
data['ID21']

{10: array([ 254.97947006,  734.69681018, 1136.78197945, 1617.25057441,
        2279.83391866, 4917.8971051 , 5075.28085122, 5254.31083292,
        5547.92836789, 6110.66741713, 6456.64458127, 7161.72798485,
        7783.58259054, 7973.84009053, 8238.98459142, 8656.43432162,
        9072.736225  , 9391.6036654 , 9776.70337184, 9870.5906888 ,
        9984.67286566]),
 2: array([1342.88026835, 1379.72355977, 1401.96681609, 1421.00733797,
        2826.30647266, 4335.71123098, 5058.82184344, 6549.10037237,
        6951.68951254, 7228.04209739, 7389.2152483 , 7436.27765573,
        7451.31919931, 7824.30189618, 8068.4609527 , 9035.70301361]),
 8: array([ 254.97764642,  735.15051747, 1136.77497172, 1617.23778655,
        2279.6559335 , 4917.89030583, 5075.11894604, 5254.26082527,
        5547.92023861, 6110.66691164, 6456.63493772, 7161.72534129,
        7783.57683629, 7973.66999344, 8238.98053068, 8656.43436694,
        9072.54636383, 9391.61793462, 9774.85320296, 9870.12759134,
        998

## convert log files

In [4]:
data_dir = '../data/morpho/nInputs_pscratch_logs/'
data_files = os.listdir(data_dir)

with open(f'scripts/nInputs_dSQ_joblist.txt', 'r') as fin:
    joblist = fin.readlines()
n_by_job_ind = [int(job.split(' ')[10]) for job in joblist]
ns = list(set(n_by_job_ind))
print(ns)

In [151]:
def correct_file(path2file):
    with open(path2file, 'r') as fin:
        lines = fin.readlines()
    _lines = [line for line in lines if len(line.strip()) > 0] # remove empty lines
    _lines = [line for line in _lines if line[:2] != ': '] # remove nst lines
    writing_line = [i for i, line in enumerate(_lines) if line == 'writing results to file\n'][0]
    new_line = _lines[writing_line - 1].strip() + _lines[writing_line + 1]
    _lines = _lines[:writing_line - 1] + [new_line] + _lines[writing_line + 2:]
    with open(path2file, 'w') as fout:
        for line in _lines:
            fout.write(line)

In [153]:
bad_files = ['dsq-nInputs_dSQ_joblist-5437889_179-r813u15n12.out']

data = {n:{} for n in ns}

for data_file in data_files:
    if data_file not in bad_files:
        job_ind = int(data_file.split('-')[2].split('_')[-1])
        segment_file = lines[3].strip()
        spikes = []
        stims = {seg_loc:[] for seg_loc in range(1,31)}
        
        try: 
            with open(f'{data_dir}{data_file}', 'r') as fin:
                lines = fin.readlines()
            for line in lines:
                if 'SPIKE' in line:
                    spikes.append(float(line.split(':')[1]))
                if 'stim -' in line:
                    seg_loc = int(line.split(':')[1].strip(' t'))
                    stim_time = float(line.strip().split(':')[-1])
                    stims[seg_loc].append(stim_time)
        except:
            print(f'correcting {data_file}')
            correct_file(f'{data_dir}{data_file}')
            
            with open(f'{data_dir}{data_file}', 'r') as fin:
                lines = fin.readlines()
            for line in lines:
                if 'SPIKE' in line:
                    spikes.append(float(line.split(':')[1]))
                if 'stim -' in line:
                    seg_loc = int(line.split(':')[1].strip(' t'))
                    stim_time = float(line.strip().split(':')[-1])
                    stims[seg_loc].append(stim_time)
                
        # remove duplicate spikes
        _spikes = [spikes[0]]
        for i in range(1,len(spikes)):
            if spikes[i] - spikes[i-1] > 1:
                _spikes.append(spikes[i])
        
        data[n_by_job_ind[job_ind]][data_file] = {'spikes':_spikes, 'stims':stims}